In [1]:
import requests
import pandas as pd

from datetime import datetime, timedelta
import time
import json
import re

import sqlite3

In [2]:
def convert_date(date_str):
    year = int(date_str[:3]) + 1911
    month = int(date_str[4:6])
    day = int(date_str[7:9])
    return f"{year}-{month:02d}-{day:02d}"

def save_to_database(df, db_name="stocks.db"):
    conn = sqlite3.connect(db_name)
    df.to_sql("stocks", conn, if_exists="replace", index=False)
    conn.close()
# from 1 to 13
refs = {
    "第一款": 1,
    "第二款": 2,
    "第三款": 3,
    "第四款": 4,
    "第五款": 5,
    "第六款": 6,
    "第七款": 7,
    "第八款": 8,
    "第九款": 9,
    "第十款": 10,
    "第十一款": 11,
    "第十二款": 12,
    "第十三款": 13
}

In [3]:
def convert_date(date_str):
    try:
        # date_str = date_str.replace("~", "～")  # 半形轉全形
        start_date, end_date = date_str.split("～")
        
        start_year = int(start_date[:3]) + 1911
        start_month = int(start_date[4:6])
        start_day = int(start_date[7:9])
        
        end_year = int(end_date[:3]) + 1911
        end_month = int(end_date[4:6])
        end_day = int(end_date[7:9])
        return f"{start_year}-{start_month:02d}-{start_day:02d}～{end_year}-{end_month:02d}-{end_day:02d}"
    except Exception as e:
        print(f"❌ Error converting: {repr(date_str)} -> {e}")
        return None   # 或者直接回傳原始字串


In [20]:
datas=[]
with open("TSE_punished.json", "r", encoding="utf-8") as f:
    TSE_data = json.load(f)
    # datas.append(TSE_data)
with open("OTC_punished.json", "r", encoding="utf-8") as f:
    OTC_data = json.load(f)
    datas.append(OTC_data)

for idx, data_source in enumerate(datas):
    if data_source == TSE_data:
        data = pd.DataFrame(data_source["data"], columns=data_source["fields"])
        data.drop(columns=['處置措施','處置內容','備註'], inplace=True)
        data['Source'] = 'TSE'
    if data_source == OTC_data:
        data = pd.DataFrame(data_source['tables'][0]["data"], columns=data_source['tables'][0]["fields"])
        data.drop(columns=['收盤價','本益比',' ','處置內容'], inplace=True)
        data.rename(columns={
            "處置原因": "處置條件",
            "處置起訖時間": "處置起迄時間"
        }, inplace=True)
        data['Source'] = 'OTC'
        data=data[['編號', '公布日期', '證券代號', '證券名稱', '累計', '處置條件', '處置起迄時間', 'Source']]
    data['處置起迄時間'] = data['處置起迄時間'].str.replace("~", "～")  # 半形轉全形
    
    data['處置起迄時間']=data['處置起迄時間'].apply(convert_date)

    data[['處置起始時間', '處置結束時間']] = data['處置起迄時間'].str.split('～', n=1, expand=True)
    data['證券名稱'] = data['證券名稱'].str.split('(').str.get(0)
    data=data.sort_values(by="處置結束時間", ascending=True)
data



,編號,公布日期,證券代號,證券名稱,累計,處置條件,處置起迄時間,Source,處置起始時間,處置結束時間
0,1,114/08/18,1815,富喬,1,因連續3個營業日達本中心作業要點第四條第一項第一款(./attention.html),2025-08-19～2025-09-01,OTC,2025-08-19,2025-09-01
22,22,114/08/18,6895,宏碩系統,1,因連續3個營業日達本中心作業要點第四條第一項第一款(./attention.html),2025-08-19～2025-09-01,OTC,2025-08-19,2025-09-01
20,20,114/08/18,6727,亞泰金屬,1,因連續3個營業日達本中心作業要點第四條第一項第一款(./attention.html),2025-08-19～2025-09-01,OTC,2025-08-19,2025-09-01
10,10,114/08/18,4577,達航科技,1,最近10個營業日內有6個營業日,2025-08-19～2025-09-01,OTC,2025-08-19,2025-09-01
6,7,114/08/18,3363,上詮,1,連續5個營業日,2025-08-19～2025-09-01,OTC,2025-08-19,2025-09-01
24,24,114/08/18,8358,金居,1,因連續3個營業日達本中心作業要點第四條第一項第一款(./attention.html),2025-08-19～2025-09-01,OTC,2025-08-19,2025-09-01
1,2,114/08/18,18156,富喬六,1,轉(交)換公司債之標的證券經本中心或臺灣證券交易所發布處置,2025-08-19～2025-09-01,OTC,2025-08-19,2025-09-01
2,3,114/08/18,3071,協禧,1,因連續3個營業日達本中心作業要點第四條第一項第一款(./attention.html),2025-08-19～2025-09-01,OTC,2025-08-19,2025-09-01
15,15,114/08/19,5475,德宏,1,因連續3個營業日達本中心作業要點第四條第一項第一款(./attention.html),2025-08-20～2025-09-02,OTC,2025-08-20,2025-09-02
16,16,114/08/19,6418,詠昇,1,因連續3個營業日達本中心作業要點第四條第一項第一款(./attention.html),2025-08-20～2025-09-02,OTC,2025-08-20,2025-09-02


In [ ]:
conn = sqlite3.connect("punished_stocks.db")
conn.row_factory = sqlite3.Row
cur = conn.cursor()
cur.execute(f"""
            SELECT `證券代號`, `證券名稱`, `處置起始時間`, `處置結束時間`, `source` FROM stocks
            WHERE 1=1 
            ORDER BY `處置結束時間` ASC
            """
        )
punished_stocks = {row["證券代號"] : {'證券名稱' : row['證券名稱'],
                                    'source' : row['source'],
                                    '處置起始時間' : row['處置起始時間'],
                                    '處置結束時間' : row['處置結束時間']} for row in cur.fetchall()}
punished_stocks

In [32]:
conn = sqlite3.connect("stock_info.db")
cursor = conn.cursor()
cursor.execute(f"""
            SELECT `type`
            FROM taiwan_stock_info
            WHERE `stock_id` = 5438
            """
)
source = cursor.fetchone()[0]
conn.close()
source


'tpex'

In [ ]:
from FinMind.data import DataLoader
import sqlite3
from datetime import datetime
import pandas as pd
# https://finmind.github.io/
# 初始化
api = DataLoader()


2025-09-08 10:33:42.133 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanStockMarketValue, data_id: 2330
2025-09-08 10:33:42.365 | ERROR    | FinMind.utility.request:request_get:62 - {'dataset': <Dataset.TaiwanStockMarketValue: 'TaiwanStockMarketValue'>, 'data_id': '2330', 'securities_trader_id': '', 'stock_id': '', 'start_date': '2023-01-01', 'end_date': '2024-01-01', 'user_id': '', 'password': '', 'device': 'package'}


Exception: {"msg":"Your level is free. Please update your user level. Detail information:https://finmindtrade.com/analysis/#/Sponsor/sponsor","status":400}

,date,stock_id,dividend_yield,PER,PBR
0,2025-01-02,2455,1.40,40.08,9.10
1,2025-01-03,2455,1.39,40.33,9.16
2,2025-01-06,2455,1.38,40.59,9.21
3,2025-01-07,2455,1.32,42.37,9.62
4,2025-01-08,2455,1.32,42.49,9.65
...,...,...,...,...,...
161,2025-09-01,2455,2.05,61.33,10.19
162,2025-09-02,2455,2.10,59.77,9.94
163,2025-09-03,2455,2.09,60.16,10.00
164,2025-09-04,2455,2.10,59.77,9.94
